In [ ]:
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
import sys
sys.path.append('/kaggle/input/kaggleword2vecutility')
from KaggleWord2VecUtility import KaggleWord2VecUtility
import pandas as pd
import numpy as np

if __name__ == '__main__':
    train = pd.read_csv(
        '/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv/labeledTrainData.tsv',
        header=0, delimiter="\t", quoting=3
    )
    test = pd.read_csv(
        '/kaggle/input/word2vec-nlp-tutorial/testData.tsv/testData.tsv',
        header=0, delimiter="\t", quoting=3
    )

    print(train.shape)
    print(train.columns.values)
    
    
    print('The first review is:')
    print(train["review"][0])

In [ ]:
    print('Download text data sets. If you already have NLTK datasets downloaded, just close the Python download window...')
    # nltk.download()  # 若需要可启用

    # 清洗训练集
    clean_train_reviews = []
    print("Cleaning and parsing the training set movie reviews...\n")
    for i in range(0, len(train["review"])):
        clean_train_reviews.append(" ".join(KaggleWord2VecUtility.review_to_wordlist(train["review"][i], True)))

    # 创建词袋模型
    print("Creating the bag of words...\n")
    vectorizer = CountVectorizer(
        analyzer="word",
        tokenizer=None,
        preprocessor=None,
        stop_words=None,
        max_features=5000
    )

    train_data_features = vectorizer.fit_transform(clean_train_reviews)
    np.asarray(train_data_features)


In [ ]:
    # 训练随机森林
    print("Training the random forest (this may take a while)...")
    forest = RandomForestClassifier(n_estimators=100, random_state=42)
    forest.fit(train_data_features, train["sentiment"])

    # 清洗测试集
    clean_test_reviews = []
    print("Cleaning and parsing the test set movie reviews...\n")
    for i in range(0, len(test["review"])):
        clean_test_reviews.append(
            " ".join(KaggleWord2VecUtility.review_to_wordlist(test["review"][i], True))
        )

    test_data_features = vectorizer.transform(clean_test_reviews)
    np.asarray(test_data_features)

    # 预测
    print("Predicting test labels...\n")
    result = forest.predict(test_data_features)

    # 输出结果
    output = pd.DataFrame(data={"id": test["id"], "sentiment": result})
    output_path = '/kaggle/working/Bag_of_Words_model.csv'
    output.to_csv(output_path, index=False, quoting=3)
    print(f"Wrote results to {output_path}")
